In [1]:
#https://github.com/wolverton-research-group/qmpy/blob/master/qmpy/data/thermodata/ssub.dat
from jarvis.db.figshare import  get_request_data
dat = get_request_data(js_tag="ssub.json",url="https://figshare.com/ndownloader/files/40084921")


Loading the zipfile...
Loading completed.


In [2]:
import pandas as pd
df = pd.DataFrame(dat)

In [3]:
df

,id,formula_energy,formula
0,465,-2.841796e+00,CaO4W
1,1197,1.766051e-09,Ho
2,1759,-3.555857e-01,PbTe
3,1454,-9.540103e-01,MoS2
4,218,-6.564038e-02,BeH2
...,...,...,...
1722,1482,-1.892842e+00,NZr
1723,773,-2.404511e+00,CrLiO2
1724,251,-4.946426e-01,BiNa3
1725,328,-7.213532e-01,Br4Ge


In [4]:
import json,zipfile
import numpy as np
path = "../../benchmarks/AI/SinglePropertyPrediction/ssub_formula_energy.json.zip"
js_tag = "ssub_formula_energy.json"
id_data = json.loads(zipfile.ZipFile(path).read(js_tag))

In [5]:
id_data.keys()

dict_keys(['train', 'test'])

In [6]:
train_ids = np.array(list(id_data['train'].keys()),dtype='int')
test_ids = np.array(list(id_data['test'].keys()),dtype='int')

In [7]:
print(len(train_ids),len(test_ids),len(train_ids)+len(test_ids))

1381 345 1726


In [8]:
train_df = df[df['id'].isin(train_ids)]
test_df = df[df['id'].isin(test_ids)]

In [9]:
len(train_df),  len(test_df)

(1381, 345)

In [10]:
%%time
from jarvis.ai.descriptors.elemental import get_element_fraction_desc
from jarvis.ai.descriptors.cfid import CFID,get_chem_only_descriptors
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

lgbm = lgb.LGBMRegressor(
    # device="gpu",
    n_estimators=1170,
    learning_rate=0.15,
    num_leaves=273,
)
X_train=[]
y_train=[]
X_test=[]
y_test=[]
train_ids=[]
test_ids=[]
for i,ii in train_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_train.append(desc)
    y_train.append(ii['formula_energy'])
    train_ids.append(ii['id'])
for i,ii in test_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_test.append(desc)
    y_test.append(ii['formula_energy'])
    test_ids.append(ii['id'])
    
X_train=np.array(X_train,dtype='float')
y_train=np.array(y_train,dtype='float')
X_test=np.array(X_test,dtype='float')
y_test=np.array(y_test,dtype='float')
lgbm.fit(X_train,y_train)
pred=lgbm.predict(X_test)
print (mean_absolute_error(y_test,pred))

0.38072016830809796
CPU times: user 12.3 s, sys: 0 ns, total: 12.3 s
Wall time: 1.18 s


# ElemNet training

In [11]:
import data_utils
from jarvis.ai.descriptors.elemental import get_element_fraction_desc
from jarvis.ai.descriptors.cfid import CFID,get_chem_only_descriptors
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

elements = ['H', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'K',
 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se',
 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In',
 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd',
 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au',
 'Hg', 'Tl', 'Pb', 'Bi', 'Ac','Th', 'Pa', 'U', 'Np', 'Pu']

def formula_to_ef(df):
    df['comp_dict'] = df['formula'].apply(lambda x: data_utils.parse_formula(x))
    df['comp_fractions'] = df['comp_dict'].apply(lambda x: data_utils.get_fractions(x))
    print('Check for',df[df['comp_fractions'].isnull()])
    df = df[~df['comp_fractions'].isnull()]

    for i,e in enumerate(elements):
        df[e] = [ x[i] for x in df['comp_fractions']]

    X_df = np.array(df[elements],dtype='float')    
    y_df = np.array(df['formula_energy'],dtype='float')
    list_df = list(df['id'])

    return X_df, y_df, list_df

In [12]:
X_train, y_train, train_ids = formula_to_ef(train_df)

X_test, y_test, test_ids = formula_to_ef(test_df)

Check for         id  formula_energy formula    comp_dict comp_fractions
48     713    1.788158e-09      Cm  {'Cm': 1.0}           None
398   1061    2.073041e-09      Fr  {'Fr': 1.0}           None
637   1060    1.707151e-09      Fm  {'Fm': 1.0}           None
754    907    1.695014e-09      Es  {'Es': 1.0}           None
1253  1788    1.844001e-09      Ra  {'Ra': 1.0}           None
Check for         id  formula_energy formula    comp_dict comp_fractions
1629  1768    1.615126e-09      Po  {'Po': 1.0}           None


/tmp/ipykernel_8572/2053813743.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comp_dict'] = df['formula'].apply(lambda x: data_utils.parse_formula(x))
/tmp/ipykernel_8572/2053813743.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comp_fractions'] = df['comp_dict'].apply(lambda x: data_utils.get_fractions(x))
/tmp/ipykernel_8572/2053813743.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [14]:
#Missing 5 train and 1 test samples due to run
X_train.shape,X_test.shape

((1376, 86), (344, 86))

In [15]:
import numpy as np
np.random.seed(1234567)
import tensorflow as tf
tf.random.set_seed(1234567)
import random
random.seed(1234567)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error

2023-05-19 13:21:34.981378: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-19 13:21:34.983950: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-19 13:21:34.983959: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [16]:
#Network from "ElemNet: Deep Learning the Chemistry of Materials From Only Elemental Composition"
#Link to paper: https://www.nature.com/articles/s41598-018-35934-y
in_layer = Input(shape=(X_train.shape[1],))

layer_1 = Dense(1024)(in_layer)
layer_1 = Activation('relu')(layer_1)

layer_2 = Dense(1024)(layer_1)
layer_2 = Activation('relu')(layer_2)

layer_3 = Dense(1024)(layer_2)
layer_3 = Activation('relu')(layer_3)

layer_4 = Dense(1024)(layer_3)
layer_4 = Activation('relu')(layer_4)
layer_4 = Dropout(0.2)(layer_4, training=True)

layer_5 = Dense(512)(layer_4)
layer_5 = Activation('relu')(layer_5)

layer_6 = Dense(512)(layer_5)
layer_6 = Activation('relu')(layer_6)

layer_7 = Dense(512)(layer_6)
layer_7 = Activation('relu')(layer_7)
layer_7 = Dropout(0.1)(layer_7, training=True)

layer_8 = Dense(256)(layer_7)
layer_8 = Activation('relu')(layer_8)

layer_9 = Dense(256)(layer_8)
layer_9 = Activation('relu')(layer_9)

layer_10 = Dense(256)(layer_9)
layer_10 = Activation('relu')(layer_10)
layer_10 = Dropout(0.3)(layer_10, training=True)

layer_11 = Dense(128)(layer_10)
layer_11 = Activation('relu')(layer_11)

layer_12 = Dense(128)(layer_11)
layer_12 = Activation('relu')(layer_12)

layer_13 = Dense(128)(layer_12)
layer_13 = Activation('relu')(layer_13)
layer_13 = Dropout(0.2)(layer_13, training=True)

layer_14 = Dense(64)(layer_13)
layer_14 = Activation('relu')(layer_14)

layer_15 = Dense(64)(layer_14)
layer_15 = Activation('relu')(layer_15)

layer_16 = Dense(32)(layer_15)
layer_16 = Activation('relu')(layer_16)

out_layer = Dense(1)(layer_16)

model = Model(inputs=in_layer, outputs=out_layer)

adam = optimizers.Adam(lr=0.0001)
model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=adam, metrics=['mean_absolute_error'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100, restore_best_weights=True)


2023-05-19 13:21:37.513814: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-19 13:21:37.514009: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-19 13:21:37.514068: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-05-19 13:21:37.514098: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-05-19 13:21:37.514129: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [18]:
%%time
model.fit(X_train, y_train, verbose=2, validation_data=(X_test, y_test), epochs=3000, batch_size=32, callbacks=[es])

Epoch 1/3000
43/43 - 1s - loss: 1.2052 - mean_absolute_error: 1.2052 - val_loss: 0.8632 - val_mean_absolute_error: 0.8632 - 1s/epoch - 28ms/step
Epoch 2/3000
43/43 - 1s - loss: 0.6862 - mean_absolute_error: 0.6862 - val_loss: 0.6277 - val_mean_absolute_error: 0.6277 - 559ms/epoch - 13ms/step
Epoch 3/3000
43/43 - 1s - loss: 0.5523 - mean_absolute_error: 0.5523 - val_loss: 0.5236 - val_mean_absolute_error: 0.5236 - 602ms/epoch - 14ms/step
Epoch 4/3000
43/43 - 1s - loss: 0.4620 - mean_absolute_error: 0.4620 - val_loss: 0.4793 - val_mean_absolute_error: 0.4793 - 552ms/epoch - 13ms/step
Epoch 5/3000
43/43 - 1s - loss: 0.4022 - mean_absolute_error: 0.4022 - val_loss: 0.4244 - val_mean_absolute_error: 0.4244 - 611ms/epoch - 14ms/step
Epoch 6/3000
43/43 - 1s - loss: 0.3418 - mean_absolute_error: 0.3418 - val_loss: 0.3939 - val_mean_absolute_error: 0.3939 - 543ms/epoch - 13ms/step
Epoch 7/3000
43/43 - 1s - loss: 0.3426 - mean_absolute_error: 0.3426 - val_loss: 0.3782 - val_mean_absolute_error: 

43/43 - 1s - loss: 0.1121 - mean_absolute_error: 0.1121 - val_loss: 0.2567 - val_mean_absolute_error: 0.2567 - 525ms/epoch - 12ms/step
Epoch 57/3000
43/43 - 1s - loss: 0.1135 - mean_absolute_error: 0.1135 - val_loss: 0.2544 - val_mean_absolute_error: 0.2544 - 524ms/epoch - 12ms/step
Epoch 58/3000
43/43 - 1s - loss: 0.1212 - mean_absolute_error: 0.1212 - val_loss: 0.2399 - val_mean_absolute_error: 0.2399 - 528ms/epoch - 12ms/step
Epoch 59/3000
43/43 - 1s - loss: 0.1066 - mean_absolute_error: 0.1066 - val_loss: 0.2469 - val_mean_absolute_error: 0.2469 - 524ms/epoch - 12ms/step
Epoch 60/3000
43/43 - 1s - loss: 0.1205 - mean_absolute_error: 0.1205 - val_loss: 0.2507 - val_mean_absolute_error: 0.2507 - 526ms/epoch - 12ms/step
Epoch 61/3000
43/43 - 1s - loss: 0.1091 - mean_absolute_error: 0.1091 - val_loss: 0.2342 - val_mean_absolute_error: 0.2342 - 526ms/epoch - 12ms/step
Epoch 62/3000
43/43 - 1s - loss: 0.1119 - mean_absolute_error: 0.1119 - val_loss: 0.2541 - val_mean_absolute_error: 0.25

43/43 - 1s - loss: 0.0947 - mean_absolute_error: 0.0947 - val_loss: 0.2244 - val_mean_absolute_error: 0.2244 - 527ms/epoch - 12ms/step
Epoch 112/3000
43/43 - 1s - loss: 0.0874 - mean_absolute_error: 0.0874 - val_loss: 0.2144 - val_mean_absolute_error: 0.2144 - 531ms/epoch - 12ms/step
Epoch 113/3000
43/43 - 1s - loss: 0.0860 - mean_absolute_error: 0.0860 - val_loss: 0.2236 - val_mean_absolute_error: 0.2236 - 533ms/epoch - 12ms/step
Epoch 114/3000
43/43 - 1s - loss: 0.0871 - mean_absolute_error: 0.0871 - val_loss: 0.2112 - val_mean_absolute_error: 0.2112 - 537ms/epoch - 12ms/step
Epoch 115/3000
43/43 - 1s - loss: 0.0921 - mean_absolute_error: 0.0921 - val_loss: 0.2279 - val_mean_absolute_error: 0.2279 - 563ms/epoch - 13ms/step
Epoch 116/3000
43/43 - 1s - loss: 0.0972 - mean_absolute_error: 0.0972 - val_loss: 0.2273 - val_mean_absolute_error: 0.2273 - 530ms/epoch - 12ms/step
Epoch 117/3000
43/43 - 1s - loss: 0.0888 - mean_absolute_error: 0.0888 - val_loss: 0.2180 - val_mean_absolute_error

Epoch 166/3000
43/43 - 1s - loss: 0.0784 - mean_absolute_error: 0.0784 - val_loss: 0.2098 - val_mean_absolute_error: 0.2098 - 593ms/epoch - 14ms/step
Epoch 167/3000
43/43 - 1s - loss: 0.0718 - mean_absolute_error: 0.0718 - val_loss: 0.2215 - val_mean_absolute_error: 0.2215 - 580ms/epoch - 13ms/step
Epoch 168/3000
43/43 - 1s - loss: 0.0787 - mean_absolute_error: 0.0787 - val_loss: 0.2043 - val_mean_absolute_error: 0.2043 - 576ms/epoch - 13ms/step
Epoch 169/3000
43/43 - 1s - loss: 0.0723 - mean_absolute_error: 0.0723 - val_loss: 0.2061 - val_mean_absolute_error: 0.2061 - 570ms/epoch - 13ms/step
Epoch 170/3000
43/43 - 1s - loss: 0.0731 - mean_absolute_error: 0.0731 - val_loss: 0.2013 - val_mean_absolute_error: 0.2013 - 580ms/epoch - 13ms/step
Epoch 171/3000
43/43 - 1s - loss: 0.0759 - mean_absolute_error: 0.0759 - val_loss: 0.2043 - val_mean_absolute_error: 0.2043 - 592ms/epoch - 14ms/step
Epoch 172/3000
43/43 - 1s - loss: 0.0783 - mean_absolute_error: 0.0783 - val_loss: 0.2006 - val_mean

Epoch 221/3000
43/43 - 1s - loss: 0.0685 - mean_absolute_error: 0.0685 - val_loss: 0.1909 - val_mean_absolute_error: 0.1909 - 567ms/epoch - 13ms/step
Epoch 222/3000
43/43 - 1s - loss: 0.0681 - mean_absolute_error: 0.0681 - val_loss: 0.1979 - val_mean_absolute_error: 0.1979 - 558ms/epoch - 13ms/step
Epoch 223/3000
43/43 - 1s - loss: 0.0624 - mean_absolute_error: 0.0624 - val_loss: 0.1944 - val_mean_absolute_error: 0.1944 - 566ms/epoch - 13ms/step
Epoch 224/3000
43/43 - 1s - loss: 0.0674 - mean_absolute_error: 0.0674 - val_loss: 0.2122 - val_mean_absolute_error: 0.2122 - 565ms/epoch - 13ms/step
Epoch 225/3000
43/43 - 1s - loss: 0.0618 - mean_absolute_error: 0.0618 - val_loss: 0.2038 - val_mean_absolute_error: 0.2038 - 564ms/epoch - 13ms/step
Epoch 226/3000
43/43 - 1s - loss: 0.0582 - mean_absolute_error: 0.0582 - val_loss: 0.1896 - val_mean_absolute_error: 0.1896 - 574ms/epoch - 13ms/step
Epoch 227/3000
43/43 - 1s - loss: 0.0694 - mean_absolute_error: 0.0694 - val_loss: 0.2104 - val_mean

...

Epoch 386/3000
43/43 - 1s - loss: 0.0474 - mean_absolute_error: 0.0474 - val_loss: 0.1995 - val_mean_absolute_error: 0.1995 - 593ms/epoch - 14ms/step
Epoch 387/3000
43/43 - 1s - loss: 0.0475 - mean_absolute_error: 0.0475 - val_loss: 0.1823 - val_mean_absolute_error: 0.1823 - 597ms/epoch - 14ms/step
Epoch 388/3000
43/43 - 1s - loss: 0.0502 - mean_absolute_error: 0.0502 - val_loss: 0.1843 - val_mean_absolute_error: 0.1843 - 591ms/epoch - 14ms/step
Epoch 389/3000
43/43 - 1s - loss: 0.0544 - mean_absolute_error: 0.0544 - val_loss: 0.1810 - val_mean_absolute_error: 0.1810 - 585ms/epoch - 14ms/step
Epoch 390/3000
43/43 - 1s - loss: 0.0480 - mean_absolute_error: 0.0480 - val_loss: 0.1785 - val_mean_absolute_error: 0.1785 - 589ms/epoch - 14ms/step
Epoch 391/3000
43/43 - 1s - loss: 0.0473 - mean_absolute_error: 0.0473 - val_loss: 0.1781 - val_mean_absolute_error: 0.1781 - 588ms/epoch - 14ms/step
Epoch 392/3000
43/43 - 1s - loss: 0.0516 - mean_absolute_error: 0.0516 - val_loss: 0.1854 - val_mean

Epoch 441/3000
43/43 - 1s - loss: 0.0454 - mean_absolute_error: 0.0454 - val_loss: 0.1946 - val_mean_absolute_error: 0.1946 - 630ms/epoch - 15ms/step
Epoch 442/3000
43/43 - 1s - loss: 0.0446 - mean_absolute_error: 0.0446 - val_loss: 0.1853 - val_mean_absolute_error: 0.1853 - 632ms/epoch - 15ms/step
Epoch 443/3000
43/43 - 1s - loss: 0.0418 - mean_absolute_error: 0.0418 - val_loss: 0.1814 - val_mean_absolute_error: 0.1814 - 626ms/epoch - 15ms/step
Epoch 444/3000
43/43 - 1s - loss: 0.0408 - mean_absolute_error: 0.0408 - val_loss: 0.1926 - val_mean_absolute_error: 0.1926 - 638ms/epoch - 15ms/step
Epoch 445/3000
43/43 - 1s - loss: 0.0503 - mean_absolute_error: 0.0503 - val_loss: 0.1788 - val_mean_absolute_error: 0.1788 - 622ms/epoch - 14ms/step
Epoch 446/3000
43/43 - 1s - loss: 0.0446 - mean_absolute_error: 0.0446 - val_loss: 0.1890 - val_mean_absolute_error: 0.1890 - 640ms/epoch - 15ms/step
Epoch 447/3000
43/43 - 1s - loss: 0.0481 - mean_absolute_error: 0.0481 - val_loss: 0.1844 - val_mean

43/43 - 1s - loss: 0.0412 - mean_absolute_error: 0.0412 - val_loss: 0.1896 - val_mean_absolute_error: 0.1896 - 853ms/epoch - 20ms/step
Epoch 497/3000
43/43 - 1s - loss: 0.0414 - mean_absolute_error: 0.0414 - val_loss: 0.1755 - val_mean_absolute_error: 0.1755 - 776ms/epoch - 18ms/step
Epoch 498/3000
43/43 - 1s - loss: 0.0407 - mean_absolute_error: 0.0407 - val_loss: 0.1872 - val_mean_absolute_error: 0.1872 - 768ms/epoch - 18ms/step
Epoch 499/3000
43/43 - 1s - loss: 0.0417 - mean_absolute_error: 0.0417 - val_loss: 0.1773 - val_mean_absolute_error: 0.1773 - 747ms/epoch - 17ms/step
Epoch 500/3000
43/43 - 1s - loss: 0.0392 - mean_absolute_error: 0.0392 - val_loss: 0.1823 - val_mean_absolute_error: 0.1823 - 781ms/epoch - 18ms/step
Epoch 501/3000
43/43 - 1s - loss: 0.0405 - mean_absolute_error: 0.0405 - val_loss: 0.1781 - val_mean_absolute_error: 0.1781 - 842ms/epoch - 20ms/step
Epoch 502/3000
43/43 - 1s - loss: 0.0380 - mean_absolute_error: 0.0380 - val_loss: 0.1717 - val_mean_absolute_error

Epoch 551/3000
43/43 - 1s - loss: 0.0385 - mean_absolute_error: 0.0385 - val_loss: 0.1815 - val_mean_absolute_error: 0.1815 - 541ms/epoch - 13ms/step
Epoch 552/3000
43/43 - 1s - loss: 0.0383 - mean_absolute_error: 0.0383 - val_loss: 0.1741 - val_mean_absolute_error: 0.1741 - 543ms/epoch - 13ms/step
Epoch 553/3000
43/43 - 1s - loss: 0.0355 - mean_absolute_error: 0.0355 - val_loss: 0.1750 - val_mean_absolute_error: 0.1750 - 551ms/epoch - 13ms/step
Epoch 554/3000
43/43 - 1s - loss: 0.0338 - mean_absolute_error: 0.0338 - val_loss: 0.1731 - val_mean_absolute_error: 0.1731 - 541ms/epoch - 13ms/step
Epoch 555/3000
43/43 - 1s - loss: 0.0348 - mean_absolute_error: 0.0348 - val_loss: 0.1749 - val_mean_absolute_error: 0.1749 - 542ms/epoch - 13ms/step
Epoch 556/3000
43/43 - 1s - loss: 0.0359 - mean_absolute_error: 0.0359 - val_loss: 0.1797 - val_mean_absolute_error: 0.1797 - 553ms/epoch - 13ms/step
Epoch 557/3000
43/43 - 1s - loss: 0.0404 - mean_absolute_error: 0.0404 - val_loss: 0.1780 - val_mean

Epoch 606/3000
43/43 - 1s - loss: 0.0347 - mean_absolute_error: 0.0347 - val_loss: 0.1745 - val_mean_absolute_error: 0.1745 - 667ms/epoch - 16ms/step
Epoch 607/3000
43/43 - 1s - loss: 0.0344 - mean_absolute_error: 0.0344 - val_loss: 0.1758 - val_mean_absolute_error: 0.1758 - 637ms/epoch - 15ms/step
Epoch 608/3000
43/43 - 1s - loss: 0.0356 - mean_absolute_error: 0.0356 - val_loss: 0.1874 - val_mean_absolute_error: 0.1874 - 652ms/epoch - 15ms/step
Epoch 609/3000
43/43 - 1s - loss: 0.0409 - mean_absolute_error: 0.0409 - val_loss: 0.1783 - val_mean_absolute_error: 0.1783 - 627ms/epoch - 15ms/step
Epoch 610/3000
43/43 - 1s - loss: 0.0340 - mean_absolute_error: 0.0340 - val_loss: 0.1766 - val_mean_absolute_error: 0.1766 - 632ms/epoch - 15ms/step
Epoch 611/3000
43/43 - 1s - loss: 0.0348 - mean_absolute_error: 0.0348 - val_loss: 0.1782 - val_mean_absolute_error: 0.1782 - 616ms/epoch - 14ms/step
Epoch 612/3000
43/43 - 1s - loss: 0.0362 - mean_absolute_error: 0.0362 - val_loss: 0.1780 - val_mean

In [19]:
pred=model.predict(X_test)
print(mean_absolute_error(y_test,pred))

11/11 [==============================] - 0s 5ms/step
0.17077706645516066


In [18]:
# import os
# f=open('AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv','w')
# f.write('id,target,prediction\n')
# for i,j,k in zip(test_ids,y_test,pred):
#     line=str(i)+','+str(j)+','+str(k)+'\n'
#     f.write(line)
# f.close()
# cmd = 'zip AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv.zip AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv'
# os.system(cmd)
# cmd='rm AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv'
# os.system(cmd)
